In [1]:
import pandas as pd
import xlrd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
df1 = pd.read_excel('Northern Branch trackers/Northern Division 2021 Debris Removal Tracker.xlsx')
df2 = pd.read_csv("tetra forms ASB and SA/Asbestos Tracker.csv", parse_dates=True)

In [3]:
# we have to open the file then save it with a different name as an excel. Find a way to solve this
df3= pd.read_excel("tetra forms ASB and SA/ASB_TetraForms_survey1.xlsx")

In [4]:
# we have to open the file then save it with a different name as an excel. Find a way to solve this
# df3= pd.read_excel("ASB/ASB_TetraForms1.xlsx")

In [5]:
smartsheets = df1.copy()
asbestos = df2.copy()
tetratForms = df3.copy()

In [6]:
# tetratForms.info()

In [7]:
# asbestos.loc[asbestos['APN'] =='028-261-052',['APN','Chimney']]

# Most important info

What I Check Daily is Smart Sheets vs Tracker
* Chimneys and neshaps match
* FINAL RESULTS vs Smart sheets – biggest concern

* ASB Assessment Date Match

* End of day numbers match

In [8]:
# smartsheets.info()

# asbestos set up

In [9]:
# Take out all divisions other than central.
asbestos = asbestos[asbestos['Division'].isin(['Northern Division'])]

In [10]:
# changes date column from object to a date.
asbestos['Date Collected'] = pd.to_datetime(asbestos['Date Collected']).dt.date
asbestos['Chimney Tip Date'] = pd.to_datetime(asbestos['Chimney Tip Date']).dt.date
asbestos['Chimney Abatement Completed Date'] = pd.to_datetime(asbestos['Chimney Abatement Completed Date']).dt.date
asbestos['Asbestos Abatement Completed Date'] = pd.to_datetime(asbestos['Asbestos Abatement Completed Date']).dt.date

In [11]:
asbestos.columns = map(str.upper, asbestos.columns)

In [12]:
# Column section all upper case
asb_cols = asbestos[['APN', 'FINAL RESULTS', 'CHIMNEY', 'CHIMNEY TIP DATE', 'CHIMNEY FINDING',
                 'NESHAP WALLS',
                 'DATE COLLECTED', 
                 'SAMPLE QUANTITY', 'POINT COUNT NEEDED', 'POINT COUNT RESULTS',
                 'ASBESTOS ABATEMENT COMPLETED DATE', 'CHIMNEY ABATEMENT COMPLETED DATE',
                 'PLANNING NOTES', 'DIVISION']]

In [13]:
# # Column section normal
# test1 =asbestos[['APN','Final Results','Chimney','Chimney Tip Date','Chimney Finding',
#                  'NESHAP Walls', 
#                  'Date Collected',
#                  'Sample Quantity','Point Count Needed','Point Count Results',
#                  'Asbestos Abatement Completed Date','Chimney Abatement Completed Date',
#                  'Planning Notes','Division']]
# # test1.head(5)

# Tetra Forms setup

### sheet sample_log setup

In [14]:
TF = pd.read_excel("tetra forms ASB and SA/ASB_TetraForms_survey1.xlsx", sheet_name='sample_log')

In [15]:
max_samples = TF.groupby(['apn'])[['samplecounter']].max().reset_index()


## Tetra Forms chimney tip setup
Combine chimney tip database with sample log

In [16]:
TF_tips = pd.read_excel('tetra forms ASB and SA/ASB_TetraForms_tips1.xlsx')

In [17]:
# columns needed
TF_tips= TF_tips[['apn', 'inspection_date']]

In [18]:
# rename for easy reading for later merges
TF_tips.rename(columns={'inspection_date': 'TIP INSPECTION_TF'}, inplace=True)

In [19]:
TF_tips['TIP INSPECTION_TF'] = pd.to_datetime(TF_tips['TIP INSPECTION_TF']).dt.date

In [20]:
merge1 = max_samples.merge(TF_tips,
                          left_on='apn',
                          right_on='apn',
                          how='outer')

# Tetra Forms Abatment date setup
Combine Abatment date database with merge 1 (tips and max samples)

In [21]:
TF_abate = pd.read_excel('tetra forms ASB and SA/ASB_TetraForms_abatment1.xlsx')

In [22]:
# columns needed
TF_abate = TF_abate[['apn', 'abatement_end']]

In [23]:
TF_abate['abatement_end'] = pd.to_datetime(TF_abate['abatement_end']).dt.date

In [24]:
# rename for easy reading on later merges
TF_abate.rename(columns={'abatement_end': 'ABATEMENT END_TF'}, inplace=True)

In [25]:
merge2 = merge1.merge(TF_abate,
                     left_on='apn',
                     right_on='apn',
                     how='outer')

# Master Tetra Forms setup from all thre databases
* Chimney tips
* abatement dates
* asbestos survey info

In [26]:
tetratForms = tetratForms.merge(merge2,
                 left_on='apn',
                 right_on='apn',
                 how='outer')

In [27]:
# columns needed
tetratForms = tetratForms[['apn', 'chimneys','walls','inspection_date', 'county','samplecounter','TIP INSPECTION_TF',
                           'ABATEMENT END_TF' ]]

In [28]:
# change date object to date time format
tetratForms['inspection_date'] = pd.to_datetime(tetratForms['inspection_date']).dt.date

In [29]:
# easier read for later merges
tetratForms.rename(columns={'inspection_date': 'survey inspection date'}, inplace=True)

In [30]:
tetratForms = tetratForms[tetratForms['county'].isin(['LASSEN', 'PLUMAS', 'SISKIYOU', 'TRINITY'])]

In [31]:
# tetratForms.rename(columns={'walls': 'NESHAP Walls','inspection_date':'DATE COLLECTED'}, inplace=True)

In [32]:
# make ALL columnS upper CASE
tetratForms.columns =map(str.upper, tetratForms.columns)

In [33]:
# tetratForms.columns

# Smart Sheet set up

In [34]:
# changes date column from object to a date.
smartsheets["ASB Assessment"] = smartsheets["ASB Assessment"].dt.date
smartsheets["Chimney Tipped"] = smartsheets["Chimney Tipped"].dt.date
smartsheets["ASB Abatement"] = smartsheets["ASB Abatement"].dt.date

In [35]:
smartsheets.rename(columns={'ASB Assessment': 'Date Collected', 'Chimney Tipped':'CHIMNEY TIP DATE',
                            'County': 'SS_COUNTY'}, inplace=True)

In [36]:
smartsheets.columns = map(str.upper, smartsheets.columns)

In [37]:
# Column section normal uppercase
ss_cols = smartsheets[['APN_ROW SEGMENT','STRUCTURAL STATUS','SS_COUNTY','ASB RESULTS',
                     'CHIMNEY','CHIMNEY TIP DATE',
                     'NESHAP WALLS', 
                     "DATE COLLECTED",
                     'ASB ABATEMENT']]

In [38]:
# # Column section normal
# test2 = smartsheets[['APN_ROW Segment','ASB Results',
#                      'Chimney','Chimney Tipped',
#                      'NESHAP Walls', 
#                      "Date Collected",
#                      'ASB ABATEMENT']]

# merge mart sheets with ASB tracker

In [39]:
ASB_SS_Merge = ss_cols.merge(asb_cols,
           left_on='APN_ROW SEGMENT',
           right_on='APN',
           how='outer',
           suffixes=("_SS","_ASB"))

In [40]:
# ASB_SS_Merge.columns

# Merge merge 1 with tetra forms

In [41]:
final = ASB_SS_Merge.merge(tetratForms,
            left_on="APN_ROW SEGMENT",
            right_on='APN',
            how='outer',
            suffixes=('_ASB', '_TF'))

# set up columns for QC order

In [42]:
# final.columns

In [43]:
final['FINAL RESULTS'] = final['FINAL RESULTS'].str.replace("ND", 'NEGATIVE').str.replace("POS","POSITIVE")

In [44]:
final.set_index("APN_ROW SEGMENT", inplace=True)

In [45]:
# test1 = asbestos['APN', 'FINAL RESULTS', 'CHIMNEY', 'CHIMNEY TIP DATE', 'CHIMNEY FINDING',
#                  'NESHAP WALLS',
#                  'DATE COLLECTED', 
#                  'SAMPLE QUANTITY', 'POINT COUNT NEEDED', 'POINT COUNT RESULTS',
#                  'ASBESTOS ABATEMENT COMPLETED DATE', 'CHIMNEY ABATEMENT COMPLETED DATE',
#                  'PLANNING NOTES', 'DIVISION']

In [46]:
# test2 = smartsheets[['APN_ROW SEGMENT','Structural Status','ASB RESULTS',
#                      'CHIMNEY','CHIMNEY TIPPED',
#                      'NESHAP WALLS', 
#                      "DATE COLLECTED",
#                      'ASB Abatement']]

In [47]:
# final.columns

In [48]:
final = final[['APN_ASB','APN_TF','STRUCTURAL STATUS','SS_COUNTY',
               "DATE COLLECTED_SS",'DATE COLLECTED_ASB','SURVEY INSPECTION DATE','SAMPLE QUANTITY','SAMPLECOUNTER',
               'ASB RESULTS','FINAL RESULTS','POINT COUNT NEEDED', 'POINT COUNT RESULTS',
               'CHIMNEY_SS','CHIMNEY_ASB','CHIMNEYS',
               'CHIMNEY TIP DATE_SS','CHIMNEY TIP DATE_ASB','TIP INSPECTION_TF',
               'CHIMNEY FINDING', 'CHIMNEY ABATEMENT COMPLETED DATE',
               'ASB ABATEMENT','ASBESTOS ABATEMENT COMPLETED DATE', 'ABATEMENT END_TF',
               'NESHAP WALLS_SS','NESHAP WALLS_ASB','WALLS',
               'DIVISION']]

In [49]:
# # add this once we have a Tetra form part done
# final = final[['APN_ASB','APN_TF',
#                'ASB Results','Final Results',
#                'Chimney_SS','Chimney_ASB','CHIMNEYS',
#                'NESHAP Walls_SS','NESHAP Walls_ASB','WALLS',
#        "ASB Assessment",'Date Collected','INSPECTION_DATE', 'Division']]

In [50]:
final.to_excel("Northern Dixie ASB Audit.xlsx")

# Get number of ssamples taken compared to number of samples positive.
### this is extra if I have time.

In [51]:
# smartsheets
# asbestos
# tetratForms 